## Building Algorithm 9.1

In [17]:
using UncNLPTestSet, TRS, LinearAlgebra
nlp = SelectProgram("SROSENBR")
adjdim!(nlp, 200);

┌ Warning: adjdim!: This operation may be unstable
└ @ UncNLPTestSet /Users/daniel/.julia/dev/UncNLPTestSet/src/UncNLPTestSet.jl:133


In [4]:
BLAS.set_num_threads(4)

function qntrHS(nlp, x₀, Hᵢ, Δₖ = 1.0, b=4, ϵ=10e-6)
	η = 0.1
	Δₘ = 1 # out of thin air 
	n = nlp.n
	xₖ = x₀
	fₖ = obj(nlp, xₖ)
	Sₖ = orth(rand(n, 2b-1))
	gₖ, hₖ, Yₖ = gHS(nlp, xₖ, Sₖ)
	Hₖ = BFGS(Hᵢ, [Sₖ gₖ], [Yₖ hₖ], ϵ)
	Hₖ = Symmetric(Hₖ) # this tage may not be needed

	k = 0
	while norm(gₖ) ≥ ϵ  # confirm norm(gₖ).. this may be computed down below 
		Qₖ = [hₖ gₖ Yₖ]  			

		# trust-region subproblem
		P = Symmetric(Qₖ'*Hₖ*Qₖ)
		b = Qₖ'*Hₖ*gₖ
		C = Symmetric(Qₖ'*Hₖ^2*Qₖ)
		aₖ, _ = trs_small(P, b, Δₖ, C)
		aₖ = vec(aₖ) # update TRS ... it returns a column matirx

		# Handle result, i.e. step must satisfy convergence criterion
		q = Qₖ*aₖ 
		pₖ = vec(Hₖ*q)
		xₜ = xₖ + pₖ # t for trial
		fₜ = obj(nlp, xₜ) # t for trial
		ρ = (fₜ - fₖ)/(0.5*aₖ'*P*aₖ + b'*aₖ) # confirm no mₖ(0) in denominator. and bubble up minus to swap terms in numerator

		# Adjust tr-radius check 
		if ρ < η
			Δₖ = 0.25*Δₖ # should I safe gaurd this to be above a lower bound
		elseif ρ > 0.75 && norm(pₖ) ≈ Δₖ
			Δₖ = min(2Δₖ, Δₘ)
		end

		# Update if we took the step?
		if ρ > η
			xₖ = xₖ .+ pₖ
			fₖ = fₜ
			Sₖ = orth(Yₖ - Sₖ*(Sₖ'Yₖ))
			gₖ, hₖ, Yₖ = gHS(nlp, xₖ, Sₖ)
			Hₖ = BFGS(Hᵢ, [Sₖ gₖ], [Yₖ hₖ], ϵ)
		end
		
		# a safeguard
		k+=1
		if k > 50000
			println("Number of iterations: $k")
			return xₖ, gₖ
		end
	end
	println("Number of iterations: $k")
	return xₖ, gₖ
end

qntrHS (generic function with 4 methods)

In [10]:
H0 = rand(nlp.n, nlp.n)
H0 = Symmetric((H0 + H0'))

xₘ, ∇fxₘ = qntrHS(nlp, nlp.x0, H0);

Number of iterations: 50001


In [12]:
[xₘ ∇fxₘ]

200×2 Matrix{Float64}:
 0.515402    33.7069
 0.0974398  -33.6398
 0.568391    42.5947
 0.131925   -38.2288
 0.503948    40.1791
 0.04972    -40.8486
 0.65324     97.8182
 0.0497099  -75.4024
 0.399516    13.4241
 0.0680958  -18.3035
 0.516664    45.8279
 0.0405149  -45.2853
 0.479761    27.8869
 ⋮          
 0.684675    52.6434
 0.274256   -38.9047
 0.619837    -0.407017
 0.382772    -0.285002
 0.596745     5.02356
 0.33168     -4.88489
 0.711823    36.0238
 0.378149   -25.7087
 0.645958    40.5126
 0.257729   -31.9066
 0.610239    26.6882
 0.259863   -22.5057

In [11]:
norm(∇fxₘ)

631.6463784879031

It seems to have worked very poorly. Things work much better when H0 is the identity

In [7]:
H0 = zeros(nlp.n, nlp.n) + I
H0 = Symmetric(H0)

xₘ, ∇fxₘ = qntrHS(nlp, nlp.x0, H0);

Number of iterations: 14782


In [8]:
[xₘ ∇fxₘ]

200×2 Matrix{Float64}:
 1.0  -5.52412e-7
 1.0   8.27981e-7
 1.0  -6.60902e-8
 1.0   5.85298e-7
 1.0  -3.5623e-7
 1.0   7.30076e-7
 1.0  -9.68121e-8
 1.0   6.00638e-7
 1.0  -6.0001e-7
 1.0   8.51729e-7
 1.0  -2.64985e-7
 1.0   6.84542e-7
 1.0  -5.40067e-7
 ⋮    
 1.0  -3.83831e-7
 1.0   7.43848e-7
 1.0  -1.89769e-8
 1.0   5.61787e-7
 1.0  -3.58296e-7
 1.0   7.31103e-7
 1.0  -8.2249e-7
 1.0   9.62735e-7
 1.0   7.84632e-8
 1.0   5.13162e-7
 1.0  -1.9056e-7
 1.0   6.47421e-7

In [9]:
norm(∇fxₘ)

8.572936957455815e-6